# ICU Data Sorter

### Creation

In [1]:
import pandas as pd
import numpy as np
from os.path import exists
#import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

general_desc = ["RecordID","Age","Gender","Height","ICUType","Weight"]
ts_desc = ["Albumin","ALP","ALT","AST","Bilirubin","BUN","Cholesterol",
           "Creatinine","DiasABP","FiO2","GCS","Glucose","HCO3","HCT","HR","K",
          "Lactate","Mg","MAP","MechVent","Na","NIDiasABP","NIMAP","NISysABP",
          "PaCO2","PaO2","pH","Platelets","RespRate","SaO2","SysABP","Temp",
           "TroponinI","TroponinT","Urine","WBC"] #weight as time series ignored
ts_full = []
for each in ts_desc:
    for end in [".count",".min",".mean",".median",".max",".first",".last"]:
        new = each + end
        ts_full.append(new)
        
variables = general_desc + ts_full

ICU_df = pd.DataFrame(columns=variables)


In [19]:
ICU_df

,RecordID,Age,Gender,Height,ICUType,Weight,Albumin.count,Albumin.min,Albumin.mean,Albumin.median,...,Urine.max,Urine.first,Urine.last,WBC.count,WBC.min,WBC.mean,WBC.median,WBC.max,WBC.first,WBC.last
0,142675.0,70.0,1.0,175.3,2.0,85.00,3.0,1.7,2.133333,2.0,...,1705.0,1030.0,100.0,10.0,4.8,6.760000,6.35,8.7,7.9,6.1
1,142676.0,57.0,0.0,-1.0,3.0,-1.00,0.0,-1.0,-1.000000,-1.0,...,1550.0,1550.0,180.0,2.0,15.1,18.750000,18.75,22.4,22.4,15.1
2,142680.0,60.0,1.0,182.9,3.0,78.80,0.0,-1.0,-1.000000,-1.0,...,600.0,120.0,500.0,3.0,12.8,14.966667,14.50,17.6,14.5,17.6
3,142683.0,64.0,1.0,180.3,2.0,145.20,2.0,1.6,2.300000,2.3,...,450.0,60.0,50.0,8.0,16.9,20.487500,20.45,24.2,18.3,18.3
4,142688.0,51.0,1.0,172.7,4.0,90.00,0.0,-1.0,-1.000000,-1.0,...,300.0,140.0,60.0,2.0,12.5,14.250000,14.25,16.0,16.0,12.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3888,152848.0,64.0,0.0,-1.0,4.0,70.00,0.0,-1.0,-1.000000,-1.0,...,1570.0,1570.0,80.0,2.0,11.2,14.250000,14.25,17.3,17.3,11.2
3889,152849.0,78.0,1.0,180.3,2.0,83.18,0.0,-1.0,-1.000000,-1.0,...,800.0,120.0,80.0,3.0,14.5,16.766667,15.80,20.0,20.0,15.8
3890,152851.0,90.0,1.0,177.8,3.0,87.00,0.0,-1.0,-1.000000,-1.0,...,70.0,0.0,35.0,7.0,18.0,28.071429,27.40,41.8,27.4,21.4
3891,152862.0,49.0,0.0,-1.0,3.0,57.00,0.0,-1.0,-1.000000,-1.0,...,600.0,400.0,65.0,2.0,13.6,15.100000,15.10,16.6,13.6,16.6


In [3]:
n = 142674
for file in range(10191):
    current_patient = []
    n += 1
    if exists(str(n)+".txt"):
        convert = pd.read_csv(str(n)+".txt")
        for desc in general_desc:#non time series
            temp = convert.loc[convert['Parameter'] == desc,'Value']
            if desc != "Weight":
                current_patient.append(float(temp))
            else:
                temp = convert.loc[convert['Parameter'] == desc,['Value']]
                current_patient.append(float(temp['Value'].iat[0]))
        for desc in ts_desc:#time series variables
            temp = convert.loc[convert['Parameter'] == desc,'Value']
            current_patient.append(int(len(temp.index)))#time series counts
            if len(temp.index) != 0:
                current_patient.append(float(temp.min()))#time series min
            else:
                current_patient.append(-1)
            if len(temp.index) != 0:
                current_patient.append(float(temp.mean()))#time series mean
            else:
                current_patient.append(-1)
            if len(temp.index) != 0:
                current_patient.append(float(temp.median()))#time series median
            else:
                current_patient.append(-1)
            if len(temp.index) != 0:
                current_patient.append(float(temp.max()))#time series max
            else:
                current_patient.append(-1)
            temp = convert.loc[convert['Parameter'] == desc,['Value']]
            if int(len(temp.index)) != 0:
                current_patient.append(float(temp['Value'].iat[0]))#time series first
                current_patient.append(float(temp['Value'].iat[-1]))#time series last
            else:
                current_patient.append(-1)
                current_patient.append(-1)
        ICU_df.loc[len(ICU_df.index)] = current_patient
#print(current_patient)
#print(len(current_patient))
#print(len(ts_full)+len(general_desc))

print("Complete")

Complete


#### UPLOAD

In [4]:
ICU_df.to_csv("ICU.csv",index=False)

### Further Processing

#### Height

In [28]:
ICU_df2 = pd.read_csv('ICU.csv')

In [29]:
HeightEstimate = ICU_df2[["Age","Gender","Height"]]
HeightEstimate = HeightEstimate.replace(-1,np.NaN)

In [30]:
imputer = IterativeImputer(estimator=RandomForestRegressor())
imputer.fit(HeightEstimate)

D:\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


IterativeImputer(estimator=RandomForestRegressor())

In [31]:
#HeightEstimate
display(pd.DataFrame(imputer.transform(HeightEstimate)))

,0,1,2
0,70.0,1.0,175.300000
1,57.0,0.0,163.560077
2,60.0,1.0,182.900000
3,64.0,1.0,180.300000
4,51.0,1.0,172.700000
...,...,...,...
3888,64.0,0.0,160.958000
3889,78.0,1.0,180.300000
3890,90.0,1.0,177.800000
3891,49.0,0.0,162.764116


In [32]:
height = pd.DataFrame(imputer.transform(HeightEstimate))
height = height.round(1)

In [33]:
ICU_df2['Height'] = height[2]

In [34]:
#ICU_df

#### Weight

In [35]:
WeightEstimate = ICU_df2[["Age","Gender","Height","Weight"]]
WeightEstimate = WeightEstimate.replace(-1,np.NaN)
imputer = IterativeImputer(estimator=RandomForestRegressor())
imputer.fit(WeightEstimate)

D:\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


IterativeImputer(estimator=RandomForestRegressor())

In [36]:
display(pd.DataFrame(imputer.transform(WeightEstimate)))

,0,1,2,3
0,70.0,1.0,175.3,85.000000
1,57.0,0.0,163.6,76.072324
2,60.0,1.0,182.9,78.800000
3,64.0,1.0,180.3,145.200000
4,51.0,1.0,172.7,90.000000
...,...,...,...,...
3888,64.0,0.0,161.0,70.000000
3889,78.0,1.0,180.3,83.180000
3890,90.0,1.0,177.8,87.000000
3891,49.0,0.0,162.8,57.000000


In [37]:
weight = pd.DataFrame(imputer.transform(WeightEstimate))
weight = weight.round(1)

In [38]:
ICU_df2['Weight'] = weight[3]

#### In hospital death

In [62]:
ICU_deathstats = pd.read_csv('Outcomes-b.txt')

In [63]:
ICU_deathstats_improved = ICU_df2.join(ICU_deathstats.set_index('RecordID'), on='RecordID')

In [64]:
ICU_df2["Death"] = ICU_deathstats_improved["In-hospital_death"]
#ICU_df2

,RecordID,Age,Gender,Height,ICUType,Weight,Albumin.count,Albumin.min,Albumin.mean,Albumin.median,...,Urine.first,Urine.last,WBC.count,WBC.min,WBC.mean,WBC.median,WBC.max,WBC.first,WBC.last,Death
0,142675.0,70.0,1.0,175.3,2.0,85.0,3.0,1.7,2.133333,2.0,...,1030.0,100.0,10.0,4.8,6.760000,6.35,8.7,7.9,6.1,1
1,142676.0,57.0,0.0,163.6,3.0,76.1,0.0,-1.0,-1.000000,-1.0,...,1550.0,180.0,2.0,15.1,18.750000,18.75,22.4,22.4,15.1,0
2,142680.0,60.0,1.0,182.9,3.0,78.8,0.0,-1.0,-1.000000,-1.0,...,120.0,500.0,3.0,12.8,14.966667,14.50,17.6,14.5,17.6,1
3,142683.0,64.0,1.0,180.3,2.0,145.2,2.0,1.6,2.300000,2.3,...,60.0,50.0,8.0,16.9,20.487500,20.45,24.2,18.3,18.3,0
4,142688.0,51.0,1.0,172.7,4.0,90.0,0.0,-1.0,-1.000000,-1.0,...,140.0,60.0,2.0,12.5,14.250000,14.25,16.0,16.0,12.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3888,152848.0,64.0,0.0,161.0,4.0,70.0,0.0,-1.0,-1.000000,-1.0,...,1570.0,80.0,2.0,11.2,14.250000,14.25,17.3,17.3,11.2,0
3889,152849.0,78.0,1.0,180.3,2.0,83.2,0.0,-1.0,-1.000000,-1.0,...,120.0,80.0,3.0,14.5,16.766667,15.80,20.0,20.0,15.8,0
3890,152851.0,90.0,1.0,177.8,3.0,87.0,0.0,-1.0,-1.000000,-1.0,...,0.0,35.0,7.0,18.0,28.071429,27.40,41.8,27.4,21.4,0
3891,152862.0,49.0,0.0,162.8,3.0,57.0,0.0,-1.0,-1.000000,-1.0,...,400.0,65.0,2.0,13.6,15.100000,15.10,16.6,13.6,16.6,0


#### Upload

In [65]:
ICU_df2.to_csv("ICU_setb.csv",index=False)

In [50]:
#ICU_deathstats_improved